# Python Profiling

## What is profiling?

> <font size=3> The process of monitoring the execution of various parts of the software in order to find the sections that can be optimized within their performance/memory usage.</font>

The aim of profiling is to find and identify the most <b>time- or memory-consuming parts</b> (eg. functions or code sections) of the program, so called "bottlenecks".


<img src="profiling_img.png"
     alt="Profiling diagram"
     style="float: left; margin-left: 80px;" />

We classify profiling into performance and memory profiling. The former can be divided into <b>deterministic</b> and <b>statistical</b>. In this presentation we'll focus on deterministic approach, and a bit on the statistical one.

### Deterministic vs. statistical profiling

<img src="det_stat1.png"
     alt="Profiling diagram"
     style="float: left; margin-left: 80px;" />



Deterministic profiling is a type of profiling where each function call, function return, and exception event is monitored, and the timing is made for the intervals between these events. It provides much information about the program execution, but has big impact on its performance (adds much <i>overhead</i>).

Statistical profiling, on the other hand, takes samples of the program execution state periodically at constant intervals. In other words, it checks which function(s) is being called at each time interval in order to count the relative time spent in each function during execution. This profiling type therefore has less impact on the performance than deterministic profiling.

Because of this, statistical profiling is better suited for larger production systems, whereas deterministic is more useful for smaller programs.

In this tutorial, we will pay attention to the following tools:
 - time
 - cProfile, profile
 - pstats
 - snakeviz
 - gprof2dot, graphviz
 - pyinstrument
 - trace

## 1. Simple time measuring with `time`

We can measure program execution time with a simple usage of <i>time</i> module.

In [ ]:
import time

def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Start measuring
start_time = time.perf_counter()

# Calculate 30th Fibonacci number
print(fib(30))

# Stop measuring
time_elapsed = time.perf_counter() - start_time
print('Time elapsed:', time_elapsed, 's')

#### Helper Timer class

This code can be simplified with a helper Timer class defined below. It can be used when we want to measure the time many times in our code.

In [ ]:
class Timer:
    _start_time = None
    @classmethod
    def start_measure(cls):
        cls._start_time = time.perf_counter()

    @classmethod
    def print_time(cls):
        time_elapsed = time.perf_counter() - cls._start_time
        print('Time elapsed: {:.5f} s'.format(time_elapsed))
        

# Start
Timer.start_measure()

print(fib(32))
Timer.print_time()
print(fib(31))
Timer.print_time()

# Start again
Timer.start_measure()

print(fib(35))
Timer.print_time()

# 2. cProfile, profile

`cProfile` and `profile` are built-in Python modules for profiling performance of a program. They are both implementations of the same interface, however:
- <b>cProfile</b> is written in C and adds less overhead (additional execution time)
- <b>profile</b> is a pure Python module with more overhead, but can be extended. More info at https://docs.python.org/3/library/profile.html#module-cProfile.
- another library that is used along with either profiling module is <b>pstats</b> (built-in). pstats includes the <i>Stats</i> class which is used to store results from profiler. See more https://docs.python.org/3/library/profile.html?highlight=pstats#module-pstats

We will focus on <b>cProfile</b>.


In [ ]:
from math import sin
from time import sleep

def f(x):
    return 10.0 * sin(x)

def compute(n):
    x = 1
    for i in range(n):
        x = f(x)
    return x

def fun1():
    return compute(3*10**5)

def fun2():
    return compute(5*10**5)

def fun3():
    sleep_()
    return compute(7*10**5)

def sleep_():
    sleep(0.1)


def main():
    sum = fun1() + fun2() + fun3()
    sleep_()
    print('Calcuated sum: {:.3f}\n'.format(sum))

### I) cProfile.run("...")

We can run the profiler on a specific code with the cProfile.run() method. As a parameter we pass the code of our program we want to profile, eg. a function call, given in parenthesis.

In [ ]:
import cProfile

cProfile.run("main()")

<font size=4>cProfile metrics:</font>
<font size=3>

- <i>ncalls</i> - number of function calls
- <i>tottime</i> - total time spent in function itself (<u>excluding subcalls</u>)
- <i>percall</i> (1) - total time spent per call = <i>tottime/ncalls</i>
- <i>cumtime</i> - cumulative time spent in function, including cubcalls
- <i>percall</i> (2) - cumulative time per call = <i>cumtime/ncalls</i>
</font>

### II) Define the decorator

Another, pretty way to use the profiler is by defining a function decorator, which adds the functionality of profiler to it. If you want to save the results too, use the method ps.dump_states() called in the line 19.

In [ ]:
import pstats
import io

# Profiler decorator
def profile(func):

    def inner(*args, **kwargs):
        pr = cProfile.Profile()
        # Enable profiling
        pr.enable()
        # Execute the function
        ret_val = func(*args, **kwargs)
        # Disable profiling
        pr.disable()
        ios = io.StringIO()
        ps = pstats.Stats(pr, stream=ios)
        ps.print_stats()
        # Save stats
        # ps.dump_stats('output.pstats')
        print(ios.getvalue())
        # Return the function value
        return ret_val

    return inner

Having the decorator defined, add it with the annotation mark to the function definition, like below:

In [ ]:
@profile
def main_with_prof():
    sum = fun1() + fun2() + fun3()
    sleep_()
    print('Calcuated sum: {:.3f}\n'.format(sum))

Then, simply calling the function will run the profiler along with it:

In [ ]:
main_with_prof()

### III) Run from CLI

We can also use cProfile module from the command line interface, by passing our module name as a parameter. To try it, use the module <b>sample.py</b> from this repository:

#### `python -m cProfile sample.py`
#### - Sort by total time
#### `python -m cProfile -s tottime sample.py` 
#### - Save the results
The <b>-o</b> option allows us to save the results in order to use them for further visualization.

#### `python -m cProfile -o sample.pstats`

## 3. Visualize function calls with snakeviz

<b>snakeviz</b> is a Python profiler viewer that runs as a web application in your browser.

Install it using pip:

In [ ]:
!pip install snakeviz

We can easily use snakeviz in a Jupyter Notebook using "cell magic", like below.

The result will be an interactive graph that depicts the execution time of each program function, but also visualizes how the time spent in it distributes into its subfunctions.

In [ ]:
import snakeviz
%load_ext snakeviz

In [ ]:
%%snakeviz
main()

In [ ]:
%%snakeviz
import re
re.compile("^[a-z]+[0-9]{3,}$")

### Run snakeviz from terminal

Snakeviz can be also run from command line interface to visualize profiler results that are stored in a file. To try it, use the previously saved file <b>sample.pstats</b>.

#### `python -m snakeviz sample.pstats`

## 4. Visualize call graph with gprof2dot & graphviz

The other 2 tools to visualize profiler stats, are gprof2dot and graphviz.

<b>gprof2dot</b> is a python module that generates call graphs in the DOT format. You can install it with
`pip install gprof2dot`

<b>graphviz</b> is an open source graph visualization software that generates graph PNG images from the DOT format. You can install it from the authors' site:
https://graphviz.org/download/

To get a call graph from the profiler results file, run the following command in your terminal:

### `gprof2dot -f pstats sample.pstats | dot -Tpng -o sample_call_graph.png`

- <font size=3> Convert profiler stats to a DOT graph &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | &nbsp;&nbsp; generate a PNG image from the DOT format </font>

## 5. Trace 
<p>
Trace is a built-in Python module that traces program's execution by each line. It prints on the standard output program lines in the exact order they were executed.

To test it, use the <b>prime_sieve.py</b> module from this repository:

### Trace execution
### `python -m trace --trace prime_sieve.py`
### Code coverage
The --count option counts the number of executions of each line in the program lifetime, and saves these stats to a file.

### `python -m trace --count prime_sieve.py`
<font size=5> --> `script.cover` </font>

## 6. Pyinstrument - statistical profiling

Pyinstrument module provides a statistical profiler to our program. Install it with `pip install pyinstrument`.

You can run it easily from command line interface:

### `pyinstrument local_search.py`

Output:

`
9.918 <module>  local_search.py:1
├─ 8.691 fit  local_search.py:83
│  └─ 8.690 local_search  local_search.py:28
│     └─ 8.670 evaluate  local_search.py:13
│        └─ 8.490 [self]
├─ 0.788 <module>  pandas\__init__.py:3
│     [979 frames hidden]  pandas, matplotlib, inspect, re, sre_...
├─ 0.263 <module>  numpy\__init__.py:106
│     [222 frames hidden]  numpy, pathlib, urllib, collections, ...
└─ 0.129 <module>  scipy\spatial\__init__.py:95
      [46 frames hidden]  scipy, numpy, multiprocessing, re, co...
`


Save results into HTML format:
### `pyinstrument -r html local_search.py`

# Sources:
### - Python profilers docs: https://docs.python.org/3.7/library/profile.html
### - snakeviz site: https://jiffyclub.github.io/snakeviz/
### - grpof2dot site: https://github.com/jrfonseca/gprof2dot
### - graphviz site: https://graphviz.org/
### - Python trace docs: https://docs.python.org/3/library/trace.html
### Profiling tutorials:
### - https://medium.com/@antoniomdk1/hpc-with-python-part-1-profiling-1dda4d172cdf
### - https://stackify.com/how-to-use-python-profilers-learn-the-basics/
### - Profiling and optimizing your Python code: https://www.youtube.com/watch?v=8qEnExGLZfY
